In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
model = ChatGroq(model="openai/gpt-oss-20b")

Create a state:

In [5]:
# create a state

class LLMState(TypedDict):

    question: str
    answer: str

Define function:

In [6]:
def llm_qa(state: LLMState) -> LLMState:

    # Extract Q from state
    question = state['question']

    # Form a prompt
    prompt = f'Answer the following question {question}'

    # Ask that Q to the LLM
    answer = model.invoke(prompt).content

    #Update the answer in the state
    state['answer'] = answer

    return state


Add node & edges:

In [7]:
# create our graph

graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa', llm_qa)

# add edges
graph.add_edge(START, 'llm_qa')
graph.add_edge('llm_qa', END)

# compile
workflow = graph.compile()

### Test

In [8]:
# execute

intial_state = {'question': 'How far is moon from the earth?'}

final_state = workflow.invoke(intial_state)

print(final_state['answer'])

The Moon is, on average, about **384,400 km** (≈ 238,900 mi) from Earth.  
- **Perigee (closest)**: ~ 356,500 km  
- **Apogee (farthest)**: ~ 406,700 km  
So the distance varies roughly from 356,500 km to 406,700 km, with an average of 384,400 km.


In [9]:
final_state

{'question': 'How far is moon from the earth?',
 'answer': 'The Moon is, on average, about **384,400\u202fkm** (≈\u202f238,900\u202fmi) from Earth.  \n- **Perigee (closest)**: ~\u202f356,500\u202fkm  \n- **Apogee (farthest)**: ~\u202f406,700\u202fkm  \nSo the distance varies roughly from 356,500\u202fkm to 406,700\u202fkm, with an average of 384,400\u202fkm.'}

Simple Alternative Method:

In [10]:
model.invoke('How far is moon from the earth?').content

'The Moon is about 384\u202f400\u202fkm (238\u202f900\u202fmi) from Earth on average.  \nAt its closest (perigee) it’s ~356\u202f500\u202fkm (221\u202f600\u202fmi) and at its farthest (apogee) ~406\u202f700\u202fkm (252\u202f600\u202fmi).'